In [1]:
import urllib.request
import xml.dom.minidom

pretty_xml_as_string = ''

def fetch_xml():
    # api key
    my_key = '7Yn%2BwtpnhhN0JX1sYiW%2Bf0lHHNeWUIlA7FwpwyGl2UOGTNu%2FAZ2IUTCQFFwOM0IvRPMJOXiAiaHmlv54y6EwHA%3D%3D'
    numOfRows = '10'
    zcode = '11'

    # request url
    url = "http://apis.data.go.kr/B552584/EvCharger/getChargerInfo?serviceKey=" \
          + my_key + "&numOfRows=" + numOfRows + "&pageNo=1&zcode=" + zcode
    request = urllib.request.Request(url)

    # request보내기
    response = urllib.request.urlopen(request)

    # 결과 코드 정의
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        dom = xml.dom.minidom.parseString(response_body.decode('utf-8'))

        global pretty_xml_as_string

        pretty_xml_as_string = dom.toprettyxml()
        print(pretty_xml_as_string)
    else:
        print("Error Code:" + rescode)

fetch_xml()

<?xml version="1.0" ?>
<response>
	<header>
		<resultCode>00</resultCode>
		<resultMsg>NORMAL SERVICE.</resultMsg>
		<totalCount>12349</totalCount>
		<pageNo>1</pageNo>
		<numOfRows>10</numOfRows>
	</header>
	<body>
		<items>
			<item>
				<statNm>종묘 공영주차장</statNm>
				<statId>ME000001</statId>
				<chgerId>01</chgerId>
				<chgerType>03</chgerType>
				<addr>서울특별시 종로구 종로 157</addr>
				<location>null</location>
				<lat>37.571076</lat>
				<lng>126.99588</lng>
				<useTime>24시간 이용가능</useTime>
				<busiId>ME</busiId>
				<bnm>환경부</bnm>
				<busiNm>환경부</busiNm>
				<busiCall>1661-9408</busiCall>
				<stat>9</stat>
				<statUpdDt>20211029085310</statUpdDt>
				<lastTsdt>20211028002722</lastTsdt>
				<lastTedt>20211028004608</lastTedt>
				<nowTsdt/>
				<powerType/>
				<output>50</output>
				<method>단독</method>
				<zcode>11</zcode>
				<parkingFree>Y</parkingFree>
				<note/>
				<limitYn>N</limitYn>
				<limitDetail/>
				<delYn>N</delYn>
				<delDetail/>
			</item>
			<item>
				<st

In [2]:
!pip install BeautifulSoup4
!pip install pandas

In [2]:
from bs4 import BeautifulSoup
import pandas

def make_item_list():
    soup = BeautifulSoup(pretty_xml_as_string)

    global ItemList
    ItemList = soup.findAll('item')

make_item_list()

In [3]:
def list_append():

    global statNmList
    global statIdList
    global chgerTypeList
    global addrList
    global latList
    global lngList

    statNmList = [] #충전소이름
    statIdList = [] #충전소ID
    chgerTypeList = [] #충전기타입
    addrList = [] #주소
    latList = [] #위도
    lngList = [] #경도

    for item in ItemList:
        try :
            statNm = item.find('statnm').text
            statId = item.find('statid').text
            chgerType = item.find('chgertype').text
            addr = item.find('addr').text
            lat = item.find('lat').text
            lng = item.find('lng').text

            statNmList.append(statNm)
            statIdList.append(statId)
            chgerTypeList.append(chgerType)
            addrList.append(addr)
            latList.append(lat)
            lngList.append(lng)

        except AttributeError as e:
            statNmList.append('오류')
            statIdList.append('오류')
            chgerTypeList.append('오류')
            addrList.append('오류')
            latList.append('오류')
            lngList.append('오류')

list_append()

In [4]:
def make_facility_data_api():

    facility_data_api = pandas.concat([ pandas.DataFrame(statNmList), pandas.DataFrame(statIdList), pandas.DataFrame(chgerTypeList),
                                   pandas.DataFrame(addrList), pandas.DataFrame(latList), pandas.DataFrame(lngList) ], axis=1)

    facility_data_api.to_csv("./facility_data_api.csv", encoding='utf-8-sig')

make_facility_data_api()

In [11]:
!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [5]:
import pymysql

In [6]:
import pickle

MYSQL_USER_DATA_SAVED_FILE = "res/mysql/userinfo"

mysql_user_info = dict({
    'user_id': 'khweb',
    'password': 'khWeb@123'
})

f = open(MYSQL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(mysql_user_info, f)
f.close()

del mysql_user_info


In [7]:

import pickle
import csv

def setup_db():

    MYSQL_USER_DATA_SAVED_FILE = "res/mysql/userinfo"

    global f
    f = open(MYSQL_USER_DATA_SAVED_FILE, 'rb')
    mysql_user_info = pickle.load(f)
    f.close()

    global db
    db = pymysql.connect(
        # localhost와 같은 아이피 주소임
        host = '127.0.0.1',
        # MySQL의 포트번호
        port = 3306,
        # 아이디
        user = mysql_user_info['user_id'],
        # 비밀번호
        passwd = mysql_user_info['password'],
        # 사용하려는 스키마
        db = 'esc_jpa_db',
        # 캐릭터 셋 설정
        charset = 'utf8'
    )

    global cursor
    cursor = db.cursor()

    f = open('facility_data_api.csv', 'r', encoding='UTF8')

    global csv_reader_facility
    csv_reader_facility = csv.reader(f)

    print(csv_reader_facility)

setup_db()

In [8]:

def save_in_db():


    initialization_sql = """truncate table charger_list"""

    cursor.execute(initialization_sql)

    db.commit()


    cnt = 0

    for row in csv_reader_facility:
        if cnt == 0:
            cnt += 1
            continue

        stat_nm = (row[1])
        stat_id = (row[2])
        chgertype_list = (row[3])
        addr_list = (row[4])
        lat_list = (row[5])
        lng_list = (row[6])


        sql = """
                insert into charger_list (stat_nm, stat_id, chgertype_list, addr_list, lat_list, lng_list) values
                (%s, %s, %s, %s, %s, %s)
            """

        cursor.execute(sql, (stat_nm, stat_id, chgertype_list, addr_list, lat_list, lng_list))

    db.commit()

    f.close()
    db.close()
    print('db 저장 완료')

save_in_db()

db 저장 완료
